In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi
from scipy.signal import find_peaks

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.PlaceCalc import PlaceCalc

In [ ]:
data = {}
place_calc = PlaceCalc(
    num_shuffles=200, threshold=0.99, dimension=1, bins_2d=None
    )
for key in h5_path_dict.keys():
    for fil in range(len(h5_path_dict[key])):
        fil_string = h5_path_dict[key][fil].as_posix()
        f = h5py.File(fil_string, 'r')
        exp_data = ExpData(f)
        data[fil_string] = {}
        spatial_info, significance_1d = place_calc.calc_place_cells(exp_data)
        data[fil_string]['spatial_info'] = spatial_info
        data[fil_string]['significance'] = significance_1d
        data[fil_string]['tuning'] = np.zeros((exp_data.num_neurs, 16)).astype(bool)
        for neur in range(exp_data.num_neurs):
            if not significance_1d[neur]: continue
            frs = []
            for wedge in (np.arange(16) + 1):
                wedge_frs = exp_data.fr[neur, exp_data.wedges == wedge]
                frs.append(wedge_frs)
            mean_frs = [np.nanmean(f) for f in frs]
            neur_tuning = np.argwhere(
                mean_frs > np.mean(mean_frs) + np.std(mean_frs)
                ).flatten()
            data[fil_string]['tuning'][neur, neur_tuning] = True
with open(pickle_dir / 'place_cells_1d.p', 'wb') as f: 
    pickle.dump(data, f)